<b>Disclaimer</b>  
В целом работа не выполнена :(  
Думал, что успею сделать за вечер, а в итоге долго провозился с установкой адаграма, да еще и безуспешно в общем-то. Поэтому прикладываю только какие-то зачатки работы, вдруг их хватит хотя бы на побалла. Больше в настолько долгий ящик я не буду откладывать (надеюсь). 

In [1]:
import pandas as pd
from lxml import html
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
%matplotlib inline

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]

    return ' '.join(words)

C:\Users\asus\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\asus\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [16]:
tfidf=TfidfVectorizer()
import adagram
#сделал модель адаграмовскую через режим совместимости 10 винды с линуксом, даже сделал модель, а потом что-то пошло не так
#в общем-то понятно что, в общем под виндой не импортируется, а времени уже нет доделывать

ImportError: No module named 'adagram'

In [26]:
data_rt = pd.read_csv('news_texts.csv', encoding = 'utf-8')
data_rt.dropna(inplace=True)
corpus = ' '.join(data_rt.content_norm)
corpus_xml = html.fromstring(open('paraphraser/paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []
f = open('corpus.txt', 'w', encoding = 'utf-8')
f.write(corpus)
f.close()
for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

<b>SVD</b>  
Берем все из тетрадки 4 семинара.

In [35]:
cv = CountVectorizer(min_df=3, max_df=0.4, max_features=1000)
X = cv.fit_transform(data_rt['content_norm'])
id2word = {i:w for i,w in enumerate(tfidf.get_feature_names())}
word2id = {w:i for i,w in id2word.items()}
svd = TruncatedSVD(200)
svd.fit(X)
id2vec_svd = svd.components_.T

<b>NMF</b>  
Берем оттуда же.

In [4]:
nmf = NMF(50)
nmf.fit(X)
id2vec_nmf = nmf.components_.T

<b>Word2Vec</b>

In [5]:
w2v = gensim.models.Word2Vec([text.split() for text in data_rt['content_norm']], size=50, sg=1)

<b>Fasttext</b>

In [6]:
fast_text = gensim.models.FastText([text.split() for text in data_rt['content_norm']], size=50, min_n=4, max_n=8)

<b>Adagram</b>

In [7]:
vm = adagram.VectorModel.load("out.pkl")
#притворимся, что этого не было

NameError: name 'adagram' is not defined

<b>Косинусные близости</b>

In [8]:
def most_similar(word, id2vec):
    similar = [id2word[i] for i in cosine_distances(id2vec[word2id[word]].reshape(1, -1), id2vec).argsort()[0][:10]]
    return similar

In [9]:
most_similar('победа', id2vec_svd)

['победа',
 'одержать',
 'победить',
 'успех',
 'выиграть',
 'добиться',
 'поражение',
 'настоящий',
 'уверенный',
 'суметь']

In [10]:
most_similar('победа', id2vec_nmf)

['победа',
 'второе',
 'одержать',
 'первое',
 'суметь',
 'три',
 'поражение',
 'четыре',
 'проиграть',
 'удаться']

In [11]:
w2v.most_similar('победа')

C:\Users\asus\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('одержать', 0.9100478291511536),
 ('победить', 0.8394150137901306),
 ('одерживать', 0.8190760016441345),
 ('выиграть', 0.81434166431427),
 ('успех', 0.8041506409645081),
 ('проигрыш', 0.7821857929229736),
 ('волевой', 0.7723301649093628),
 ('отпраздновать', 0.7673037052154541),
 ('одолеть', 0.765396237373352),
 ('поражение', 0.7642831802368164)]

In [12]:
fast_text.most_similar('победа')

C:\Users\asus\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('победно', 0.9180482029914856),
 ('побег', 0.9014240503311157),
 ('беда', 0.86541348695755),
 ('победоносец', 0.863079845905304),
 ('обед', 0.7952977418899536),
 ('плевок', 0.7758498191833496),
 ('побои', 0.7573943138122559),
 ('победитель', 0.7384722828865051),
 ('выигрыш', 0.7329931855201721),
 ('поражение', 0.7200791835784912)]

<b>Обучающая выборка и модель</b>

In [38]:
X_text_1 = svd.fit_transform(tfidf.transform(data['text_1_norm']))
X_text_2 = svd.fit_transform(tfidf.transform(data['text_2_norm']))
X_text = np.concatenate([X_text_1, X_text_2], axis=1)

In [43]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text, y, random_state=1)
clf = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_leaf=10,
                             class_weight='balanced')
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

             precision    recall  f1-score   support

         -1       0.56      0.56      0.56       629
          0       0.49      0.49      0.49       737
          1       0.36      0.35      0.36       441

avg / total       0.48      0.48      0.48      1807



In [44]:
X_text_1 = nmf.fit_transform(tfidf.transform(data['text_1_norm']))
X_text_2 = nmf.fit_transform(tfidf.transform(data['text_2_norm']))
X_text = np.concatenate([X_text_1, X_text_2], axis=1)

In [46]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text, y,random_state=1)
clf = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

             precision    recall  f1-score   support

         -1       0.49      0.56      0.52       629
          0       0.46      0.37      0.41       737
          1       0.32      0.36      0.34       441

avg / total       0.44      0.43      0.43      1807



In [52]:
def get_embedding(text, model, dim):
    text = text.split()

    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total)
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [53]:
dim = 50
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim)

C:\Users\asus\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [54]:
X_text_w2v = np.concatenate([X_text_1_w2v, X_text_2_w2v], axis=1)

In [55]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text_w2v, y,random_state=1)
clf = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

             precision    recall  f1-score   support

         -1       0.56      0.51      0.54       629
          0       0.46      0.46      0.46       737
          1       0.38      0.43      0.40       441

avg / total       0.48      0.47      0.47      1807



In [56]:
dim = 50
data['text_1_notnorm'] = data['text_1'].apply(tokenize)
data['text_2_notnorm'] = data['text_2'].apply(tokenize)

X_text_1_ft = np.zeros((len(data['text_1_notnorm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_notnorm']), dim))

for i, text in enumerate(data['text_1_notnorm'].values):
    X_text_1_ft[i] = get_embedding(text, fast_text, dim)
    
for i, text in enumerate(data['text_2_notnorm'].values):
    X_text_2_ft[i] = get_embedding(text, fast_text, dim)

C:\Users\asus\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [57]:
X_text_ft = np.concatenate([X_text_1_ft, X_text_2_ft], axis=1)

In [58]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text_ft, y,random_state=1)
clf = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

             precision    recall  f1-score   support

         -1       0.54      0.50      0.52       629
          0       0.48      0.54      0.50       737
          1       0.40      0.35      0.38       441

avg / total       0.48      0.48      0.48      1807

